In [ ]:
# Load dataset configuration
CONFIG_PATH = '../data/dataset_config.json'
assert os.path.exists(CONFIG_PATH), "Run 01_data_analysis_processing.ipynb first!"

with open(CONFIG_PATH) as f:
    config = json.load(f)

DATA_PATH = config['data_path']
MEAN, STD = config['mean'], config['std']
IMAGE_SIZE = config['image_size']

# Hyperparameters
OUTPUT_DIR = '../outputs/densenet'
MODEL_PATH = os.path.join(OUTPUT_DIR, 'model.pth')
BATCH_SIZE = 128
EPOCHS = 10
LR = 1e-3
SEED = 42

os.makedirs(OUTPUT_DIR, exist_ok=True)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

print(f"Data: {DATA_PATH}")
print(f"Normalization - Mean: {MEAN}, Std: {STD}")

In [ ]:
# Cached tensor dataset
class CachedTensorDataset(torch.utils.data.Dataset):
    def __init__(self, root, augment=False):
        dataset = datasets.ImageFolder(root)
        self.augment = augment
        self.tensors = []
        self.targets = []
        
        print(f"Caching {len(dataset)} images as tensors...")
        transform = transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(MEAN, STD)
        ])
        
        for path, target in tqdm(dataset.samples, desc="Loading"):
            img = Image.open(path).convert('RGB')
            self.tensors.append(transform(img))
            self.targets.append(target)
        
        self.tensors = torch.stack(self.tensors)
        self.targets = torch.tensor(self.targets)
        self.classes = dataset.classes
        print(f"Cached tensor shape: {self.tensors.shape}")
    
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        x = self.tensors[idx]
        if self.augment:
            if torch.rand(1) > 0.5:
                x = torch.flip(x, [2])
            if torch.rand(1) > 0.5:
                x = torch.flip(x, [1])
        return x, self.targets[idx]

train_dataset = CachedTensorDataset(os.path.join(DATA_PATH, 'train'), augment=True)
val_dataset = CachedTensorDataset(os.path.join(DATA_PATH, 'val'), augment=False)
test_dataset = CachedTensorDataset(os.path.join(DATA_PATH, 'test'), augment=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=0, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE*2, shuffle=False,
                        num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE*2, shuffle=False,
                         num_workers=0, pin_memory=True)

class_names = train_dataset.classes
num_classes = len(class_names)
print(f"\nClasses: {class_names}")
print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")

In [ ]:
# DenseNet-121 with frozen backbone
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

# Freeze backbone
for param in model.parameters():
    param.requires_grad = False

# Replace classifier head
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)

model = model.to(device)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable:,}")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
scaler = torch.amp.GradScaler(device.type) if USE_CUDA else None
amp_dtype = torch.float16 if USE_CUDA else torch.float32

def train_epoch(model, loader, criterion, optimizer):
    model.train()
    loss_sum, correct, total = 0, 0, 0
    
    for images, labels in tqdm(loader, desc='Train', leave=False):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        with torch.amp.autocast(device.type, dtype=amp_dtype, enabled=USE_CUDA):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        optimizer.zero_grad(set_to_none=True)
        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        loss_sum += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    
    return loss_sum / total, correct / total

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    loss_sum, correct, total = 0, 0, 0
    
    for images, labels in tqdm(loader, desc='Eval', leave=False):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        with torch.amp.autocast(device.type, dtype=amp_dtype, enabled=USE_CUDA):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        loss_sum += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    
    return loss_sum / total, correct / total

# Warmup
_ = model(torch.randn(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE, device=device))
if USE_CUDA:
    torch.cuda.synchronize()
elif USE_MPS:
    torch.mps.synchronize()
print("GPU initialized" if device.type != 'cpu' else "Running on CPU")

In [ ]:
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_acc = 0

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({'model_state_dict': model.state_dict(), 
                    'class_names': class_names, 'accuracy': best_acc}, MODEL_PATH)
    
    print(f"Epoch {epoch+1:02d}/{EPOCHS} | Train: {train_acc:.4f} | Val: {val_acc:.4f} | Best: {best_acc:.4f}")

print(f"\nTraining complete. Best validation accuracy: {best_acc:.4f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.plot(history['train_loss'], label='Train')
ax1.plot(history['val_loss'], label='Validation')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.set_title('Training Loss')

ax2.plot(history['train_acc'], label='Train')
ax2.plot(history['val_acc'], label='Validation')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.set_title('Training Accuracy')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_curves.png'), dpi=150)
plt.show()

## Test Set Evaluation

In [ ]:
checkpoint = torch.load(MODEL_PATH, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
print(f"Loaded best model (validation accuracy: {checkpoint['accuracy']:.4f})")

In [ ]:
test_preds, test_labels_list = [], []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc='Testing'):
        images = images.to(device, non_blocking=True)
        with torch.amp.autocast(device.type, dtype=amp_dtype, enabled=USE_CUDA):
            preds = model(images).argmax(1)
        test_preds.extend(preds.cpu().numpy())
        test_labels_list.extend(labels.numpy())

print("\nTEST SET RESULTS")
print("=" * 50)
print(classification_report(test_labels_list, test_preds, target_names=class_names))

In [ ]:
cm = confusion_matrix(test_labels_list, test_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Test Set')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix.png'), dpi=150)
plt.show()

test_acc = (np.array(test_preds) == np.array(test_labels_list)).mean()
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
print("=" * 50)
print("SUMMARY")
print("=" * 50)
print(f"Model: DenseNet-121 (frozen backbone)")
print(f"Validation Accuracy: {best_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Saved: {MODEL_PATH}")